In [ ]:
!pip install tflearn

     |████████████████████████████████| 107 kB 13.1 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=b38a42f72b982889d8b35a279fe8eb619121d4ef1f119c2e8f30fd9e96e349d2
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [ ]:
#importing all the necessary packages and libraries 
import nltk
import pandas as pd
import random as ran
import string
import numpy
import tflearn
import tensorflow as tf
import json
import pickle
from tabulate import tabulate
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from random import randrange
from nltk.corpus import stopwords

#downloading all the necessary data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
stop = stopwords.words('english')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

#defining all the methods

#method for trainig the chatbot
def createmodel(train, out_layer):
  tf.compat.v1.reset_default_graph()
  #taking the train's first string shape for the input layer and defining other
  #training layers as well
  neural_layer = tflearn.input_data(shape=[None, len(train[0])])
  neural_layer = tflearn.fully_connected(neural_layer, 16)
  neural_layer = tflearn.fully_connected(neural_layer, 8)
  neural_layer = tflearn.fully_connected(neural_layer, 8)
  neural_layer = tflearn.fully_connected(neural_layer, len(out_layer[0]), 
                                         activation="softmax")
  #using softmax as the activation function
  neural_layer = tflearn.regression(neural_layer)
  return neural_layer

#method for playing atlas game
def atlas(count):
  #reading the dataset and converting it to list
  #Dataset for countries
  #https://textlists.info/geography/countries-of-the-world/
  #Dataset for cities
  #https://simplemaps.com/data/world-cities
  l = pd.read_excel('Atlas.xlsx', sheet_name=0)
  atlas = l['NAMES'].tolist()
  limit = count
  #sorting it and lowercasing the names
  sort_A = sorted(atlas)
  a = [i.lower() for i in sort_A]

  c = 0
  #choosing a random alphabet
  ran_letter = ran.choice(string.ascii_letters)
  response_lw = ran_letter
  response_lw = response_lw.lower()
  while(limit>0):
      
      
      b = True
      
      
      if c == 0:
         
          print("Xeno: So lets start the game of Atlas with a random letter....You go first with the letter", ran_letter)
    
      while (b):
        #waiting for user to enter name of the city/country
          x = input().lower()
          
          #removing the name if its in the dataset and has been used
          if x in a and x[0] == response_lw:
              a.remove(x)
              b = False
          
          else:
              print('Xeno: Try another Name')
              b = True
              
      y = x[-1]
      #excluding q and x for starting words at the first round.
      notwanted = 'qx'
      r = [j for j in a if j.lower().startswith(y) if j not in notwanted] 
      
      #random choice for Xeno, so it can choose a name 
      response = ran.choice(r)
      response_lw = response[-1]
      response_lw = response_lw.lower()
      print("Xeno:",response)
      a.remove(response)
      c+=1
      limit = limit - 1
#method for bag of words for one hot encoding
def bag_of_words(t, words):
#initially making the bag list 0
  bag = []
  for i in range(len(words)):
    bag.append(0)

  s_words = nltk.word_tokenize(t)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  #if word is present then we add 1 to that index
  for se in s_words:
      for i, w in enumerate(words):
          if w == se:
              bag[i] = 1

  return numpy.array(bag)

# method for name extraction from string 
def nameextraction(customer_input):
    customer_name = []
    #splitting the string and after tokenisation, using POS to tag the words 
    customer_input = ' '.join([i for i in customer_input.split() if i not in 
                               stop])
    sentence = nltk.sent_tokenize(customer_input)
    sentence = [nltk.word_tokenize(sent) for sent in sentence]
    sentence = [nltk.pos_tag(sent) for sent in sentence]
    #checking the tags assigned if its Person or not
    for tagged_sentence in sentence:
        for chunk in nltk.ne_chunk(tagged_sentence):
            if type(chunk) == nltk.tree.Tree:
                if chunk.label() == 'PERSON': 
                  #if person then adding it to separate list
                    customer_name.append(' '.join([c[0] for c in chunk]))
    return customer_name

# method for splitting the list by spaces to get the first name (not firstname)
def namesplit(name):
    new_name = []
    for i in name:
      new_name.extend(i.split())  
    return new_name 

## Defining training parameters
epoch = 1000
b_size = 8
metric = True
##Stemmer initialised and language selected
stemmer = SnowballStemmer("english")

with open("intent.json") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    a_doc = []
    b_doc = []
    #going through the intent file and storing the data
    for intent in data["intent"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            a_doc.append(wrds)
            b_doc.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    
    for x, doc in enumerate(a_doc):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]
        #simple one hot encoding
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(b_doc[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)
#model training
layer = createmodel(training, output)
model = tflearn.DNN(layer)
model.fit(training, output, n_epoch=epoch, batch_size = b_size, 
          show_metric=metric)
model.save("model.tflearn")

#Converting the Menu in List to tablular format for better display
table = [['Item No', 'Name', 'Price(in GBP)'], ['1', 'Chicken Burger Meal', 8], 
         ['2', 'Vegan Burger Meal', 7], ['3', 'Mac n Cheese', 7], 
         ['4', 'Chicken Wrap', 6], ['5', 'Veggie Wrap', 5]]





Training Step: 8999  | total loss: 0.00015 | time: 0.059s
| Adam | epoch: 1000 | loss: 0.00015 - acc: 1.0000 -- iter: 64/69
Training Step: 9000  | total loss: 0.00015 | time: 0.065s
| Adam | epoch: 1000 | loss: 0.00015 - acc: 1.0000 -- iter: 69/69
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
#main- It is asking for user name, using the methods defined above and matching the intent tags
if __name__ == "__main__":
    print("Welcome to the Food Station!") 
    print("Tell me your name so we can start chatting!") 
    print("After ordering type Quit to exit")
    text = input("You: ")
    n_s = []
    if len(text) == 1:
      n_s.append(text)
    else:
      name = nameextraction(text)
      n_s = namesplit(name)
    #fixing the name bias since all Indian names cant be extracted 
    #(Eg: Kushagra is not extracted as a name)
    if not n_s:
      n_s.append("Customer")
    print("Hello", n_s[0], "To start chatting with Xeno")
    print("You can:")
    print("Greet the chat bot to start the conversation")
    
    flag = True
    while flag: 
      if flag == True:
        inp = input("You: ") 
      if inp.lower() == "quit": 
        break
      if inp.lower() == "atlas":
        break
      if inp.isdigit() and len(inp) == 5:
        print("Xeno: Great!")
        break
      #predicting which word belongs to which tag from the intent file
      results = model.predict([bag_of_words(inp, words)])
      results_index = numpy.argmax(results)
      tag = labels[results_index]
      #if tag is equal to a certain category, then the response from that section 
      #is sent.       
      for tg in data["intent"]:
          if tg['tag'] == tag:
              responses = tg['responses']
              if tag == "greeting":
                print("Xeno:",n_s[0], ran.choice(responses))
                print(">The menu")
                print(">Our Timings")
                print(">About me")
                break
    
              if tag == 'Food':
                print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))
                print('Xeno: Please Enter the order No. Due to covid, we have limited options')
                #opt = input()
                #print(random.choice(responses))
    
              if tag == 'ordering':
                print("Order",ran.choice(responses))
                u_id = input()
                       
    
              if tag == 'no id':
                print(ran.choice(responses))
                print("Xeno:Your new Customer ID is",randrange(10000, 99999))
                flag = False
            
              else:
                print(ran.choice(responses))

    print("Xeno: While you are waiting for your order, you can play a game of ATLAS with me.")
    print("Xeno: Please type yes or no....I am good at it :)")
    u = input()
    l = len(u)
    count = ran.randrange(5,8)
    if u[-1] == '!' or '.':
     u = u[:l-1]
    #asking if user wants to play atlas.
    if u.lower() == 'yes':
      print("Rules are simple: Name a city or country starting with the end letter of the previous name.")
      print("you can say each name only once.")
      atlas(count)
    else: 
      ("You can wait for your order. We are working on it!")
          
    print("Xeno: Your order will be ready shortly. Thanks for choosing Food Station!")

Welcome to the Food Station!
Tell me your name so we can start chatting!
After ordering type Quit to exit
You: Holly
Hello Customer To start chatting with Xeno
You can:
Greet the chat bot to start the conversation
You: hi
Xeno: Customer Hello! Ask me about
>The menu
>Our Timings
>About me
You: 2
Order  on the way. Your Order will be ready in 2 minutes. You can tell me your Customer Id if you have one. Just type your 5 digit no. If Not then I can assign one to you.
12345
 on the way. Your Order will be ready in 2 minutes. You can tell me your Customer Id if you have one. Just type your 5 digit no. If Not then I can assign one to you.
You: 1324
Order  on the way. Your Order will be ready in 2 minutes. You can tell me your Customer Id if you have one. Just type your 5 digit no. If Not then I can assign one to you.
12345
 on the way. Your Order will be ready in 2 minutes. You can tell me your Customer Id if you have one. Just type your 5 digit no. If Not then I can assign one to you.
You: 